[View in Colaboratory](https://colab.research.google.com/github/tcglarry/aoi/blob/master/aoi_xgb_ensenmble.ipynb)

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

aoi_aoi_test1.ipynb  TAHU PowerPoint Template.pdf
aoi_test	     TAHU PowerPoint Template - 柱狀圖 1.ods
Colab Notebooks      test for excel
PiXNet		     Untitled0.ipynb
Sample upload.txt


In [5]:
!ls

adc.json  drive  sample_data


In [0]:
!cp drive/aoi_test/train_images.zip .

In [7]:
!unzip train_images.zip

Archive:  train_images.zip
   creating: train_images/
 extracting: train_images/train_00000.png  
 extracting: train_images/train_00001.png  
 extracting: train_images/train_00002.png  
 extracting: train_images/train_00003.png  
 extracting: train_images/train_00004.png  
 extracting: train_images/train_00005.png  
 extracting: train_images/train_00006.png  
 extracting: train_images/train_00007.png  
 extracting: train_images/train_00008.png  
 extracting: train_images/train_00009.png  
 extracting: train_images/train_00010.png  
 extracting: train_images/train_00011.png  
 extracting: train_images/train_00012.png  
 extracting: train_images/train_00013.png  
 extracting: train_images/train_00014.png  
 extracting: train_images/train_00015.png  
 extracting: train_images/train_00016.png  
 extracting: train_images/train_00017.png  
 extracting: train_images/train_00018.png  
 extracting: train_images/train_00019.png  
 extracting: train_images/train_00020.png  
 extracting: train_ima

In [0]:
!cp drive/aoi_test/train.csv .

In [9]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,  img_to_array, load_img

from keras.layers import Dense, GlobalAveragePooling2D


import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Reshape, Dropout,Cropping2D,Activation
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import Callback
from keras.applications.mobilenet import MobileNet
from keras.applications.nasnet import NASNetMobile,NASNetLarge
import matplotlib.image as mpimg
from skimage import io
from xgboost.sklearn import XGBClassifier
 



Using TensorFlow backend.


In [10]:
train_df = pd.read_csv('train.csv')
print (train_df.keys())
print ( train_df['ID'].values.shape)


Index(['ID', 'Label'], dtype='object')
(2528,)


In [22]:

#final_list = train_df['ID'].values.tolist()

final_list = []
train_list = train_df['ID'].values
batch = 500
floor = int(np.floor(train_list.shape[0]/batch))
print ('floor=', floor)
for i in range(floor+1):
    if i < floor:
        final_list.append(train_list[i*batch:(i+1)*batch])
    else: 
        final_list.append(train_list[i*batch:])
a=0        
for l in final_list:
    #print (len(l))
    #print (l[-10:])
    a += len(l)
print ('total size', a)
print ('first 10 \n', final_list[0][:10], '\n\n', 'last 10\n', final_list[-1][-10:])




floor= 5
total size 2528
first 10 
 ['train_00000.png' 'train_00001.png' 'train_00002.png' 'train_00003.png'
 'train_00004.png' 'train_00005.png' 'train_00006.png' 'train_00007.png'
 'train_00008.png' 'train_00009.png'] 

 last 10
 ['train_02518.png' 'train_02519.png' 'train_02520.png' 'train_02521.png'
 'train_02522.png' 'train_02523.png' 'train_02524.png' 'train_02525.png'
 'train_02526.png' 'train_02527.png']


In [0]:
def build_model():
    inputs = Input(shape=(512,512,3))
    #inputs = Lambda(lambda x: x/275. - 1.0)(inputs)
    # create the base pre-trained model
    base_model = InceptionResNetV2(weights='imagenet',input_tensor=inputs, input_shape=(512,512,3), include_top=True)
    
    x = base_model.output
    print (x.get_shape)
    model = Model(inputs=base_model.input, outputs=x)
    model.summary()
    
    return model

In [19]:
model_1 = build_model()
# compile the model (should be done *after* setting layers to non-trainable)
model_1.compile(optimizer='Nadam', loss='categorical_crossentropy',metrics=['accuracy'])

225214464/225209952 [==============================] - 11s 0us/step
<bound method Tensor.get_shape of <tf.Tensor 'predictions/Softmax:0' shape=(?, 1000) dtype=float32>>
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 255, 255, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 255, 255, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 

In [0]:

def first_prediction(model,data):
    pred_temp = model.predict(data)
    return pred_temp



In [23]:
src = 'drive/aoi_test/'


if os.path.isfile(src+'data_xgb_train_ensemble.npy'):
  data_train_final = np.load(src+'data_xgb_train_ensemble.npy')
  print ('data loaded')

  do_data = False 
else:
  do_data = True

if do_data: 
  
  print ('starts doing aata')

  final_array =[]
  print (len(final_array))


  for j,l in enumerate(final_list):
      img_list=[]
      for i,img in enumerate(l):
          img = load_img('train_images/'+img)  # this is a PIL image
          x = img_to_array(img)
          x = preprocess_input(x)
          #print (x.shape)
          #x = cv2.cvtColor(y, cv2.COLOR_BGR2RGB)
          #x = x/127.5 -1.
          img_list.append(x)
      #print ('img_list=', len(img_list))
      testing_data = np.array(img_list)

      #print ('testing_data_shape', testing_data.shape)
      #train_data =  train_data[:,:,:,np.newaxis]
      #print (testing_data.shape)
      pred_temp = first_prediction(model_1,testing_data)

      print('batch = ', j, 'stage 1 completed', 'shape=', pred_temp.shape)
      np.save(src+'temp_'+str(j)+'.npy',pred_temp)

      print ('bacth npy saved', j)

      final_array.append(pred_temp)
      print ('afterbatch',j,'length=', len(final_array))
  final_array = np.asarray(final_array)
  print ('totally done')


starts doing aata
0
batch =  0 stage 1 completed shape= (500, 1000)
bacth npy saved 0
afterbatch 0 length= 1
batch =  1 stage 1 completed shape= (500, 1000)
bacth npy saved 1
afterbatch 1 length= 2
batch =  2 stage 1 completed shape= (500, 1000)
bacth npy saved 2
afterbatch 2 length= 3
batch =  3 stage 1 completed shape= (500, 1000)
bacth npy saved 3
afterbatch 3 length= 4
batch =  4 stage 1 completed shape= (500, 1000)
bacth npy saved 4
afterbatch 4 length= 5
batch =  5 stage 1 completed shape= (28, 1000)
bacth npy saved 5
afterbatch 5 length= 6
totally done


In [0]:



'''
final_df =pd.DataFrame()

img_list=[]
    
  
  
  
  
for i,img in enumerate(final_list):
    #print (img)
    #print (os.path.isfile('train_images/'+img))
    img = load_img('train_images/'+img)  # this is a PIL image
    x = img_to_array(img)

    x = preprocess_input(x)
    #print (x[:2,:2,:2])
    #print (x.shape)
    #x= x[:,:,::-1]
    #print (x.shape)
    #x = cv2.cvtColor(y, cv2.COLOR_BGR2RGB)
    #x = x/127.5 -1.
    img_list.append(x)
    
#print ('img_list=', len(img_list))
testing_data = np.array(img_list)
#print (testing_data[:2,:2,:2])

print ('testing_data_shape', testing_data.shape)
#print (testing_data[:2,:2,:2])
#train_data =  train_data[:,:,:,np.newaxis]
#print (testing_data.shape)
pred_temp = first_prediction(model_1,testing_data)   
print('train data flow to pre-trained model,  completed')

print (pred_temp.shape)
'''
    

In [36]:


#np.save(src+'data_final',pred_temp)
np.save(src+'data_xgb_train_ensemble.npy',final_array)
print ('file saved ')


file saved 


In [43]:
train_y =  train_df['Label'].values
print (train_y.shape)
#onehot_encoder = OneHotEncoder(sparse=False)
#train_y = train_y.reshape(len(train_y), 1)
#print (train_y.shape)
#train_label = onehot_encoder.fit_transform(train_y)


#print ('beofre', train_label.shape)
#print (train_df['Label'][:10])
#print (train_label[:10,:])

#train_label = train_label[:,np.newaxis,np.newaxis,:]

#print ('after', train_label.shape)

(2528,)
beofre (2528, 6)


In [35]:
#test_array =final_array[i] for i in range(final_array.shape) ]
final_array = np.concatenate((final_array[0],final_array[1],final_array[2],final_array[3],final_array[4],final_array[5]),axis=0)
final_array.shape

(2528, 1000)

In [0]:
train_x,val_x,train_y,val_y = train_test_split(final_array,train_y,test_size=0.2, shuffle=True)



In [45]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

xgb1.fit(train_x,train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)

In [48]:
from sklearn.metrics import accuracy_score
val_pred = xgb1.predict(val_x)

predictions = [round(value) for value in val_pred]
# evaluate predictions
accuracy = accuracy_score(val_y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))



#xgb1.evaluate(val_x,val_y)

Accuracy: 95.85%


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [74]:
print (val_pred.shape)

(506,)


In [0]:
import pickle
pickle.dump(xgb1, open(src+"data_train_xgb.pickle", "wb"))
 

In [57]:
!ls

adc.json  drive  sample_data  train.csv


In [0]:
!rm -rf train_images.zip
!rm -rf train_images


In [58]:
!cp drive/aoi_test/test_images2.zip .
!unzip test_images2.zip
!cp drive/aoi_test/test.csv .

Archive:  test_images2.zip
   creating: test_images/
 extracting: test_images/test_00000.png  
 extracting: test_images/test_00001.png  
 extracting: test_images/test_00002.png  
 extracting: test_images/test_00003.png  
 extracting: test_images/test_00004.png  
 extracting: test_images/test_00005.png  
 extracting: test_images/test_00006.png  
 extracting: test_images/test_00007.png  
 extracting: test_images/test_00008.png  
 extracting: test_images/test_00009.png  
 extracting: test_images/test_00010.png  
 extracting: test_images/test_00011.png  
 extracting: test_images/test_00012.png  
 extracting: test_images/test_00013.png  
 extracting: test_images/test_00014.png  
 extracting: test_images/test_00015.png  
 extracting: test_images/test_00016.png  
 extracting: test_images/test_00017.png  
 extracting: test_images/test_00018.png  
 extracting: test_images/test_00019.png  
 extracting: test_images/test_00020.png  
 extracting: test_images/test_00021.png  
 extracting: test_image

In [65]:



final_list = []

test_df = pd.read_csv('test.csv')
test_list = test_df['ID'].values

batch = 500
floor = int(np.floor(test_list.shape[0]/batch))
print ('floor=', floor)
for i in range(floor+1):
    if i < floor:
        final_list.append(test_list[i*batch:(i+1)*batch])
    else: 
        final_list.append(test_list[i*batch:])
a=0        
for l in final_list:
    #print (len(l))
    #print (l[-10:])
    a += len(l)
print ('total size', a)
print ('first 10 \n', final_list[0][:10], '\n\n', 'last 10\n', final_list[-1][-10:])



floor= 20
total size 10142
first 10 
 ['test_00000.png' 'test_00001.png' 'test_00002.png' 'test_00003.png'
 'test_00004.png' 'test_00005.png' 'test_00006.png' 'test_00007.png'
 'test_00008.png' 'test_00009.png'] 

 last 10
 ['test_10132.png' 'test_10133.png' 'test_10134.png' 'test_10135.png'
 'test_10136.png' 'test_10137.png' 'test_10138.png' 'test_10139.png'
 'test_10140.png' 'test_10141.png']


In [68]:
if os.path.isfile(src+'data_xgb_test_ensemble.npy'):
  data_train_final = np.load(src+'data_xgb_test_ensemble.npy')
  print ('data loaded')

  do_data = False 
else:
  do_data = True

if do_data: 
  
  print ('starts doing aata')

  final_array =[]
 


  for j,l in enumerate(final_list):
      img_list=[]
      for i,img in enumerate(l):
          img = load_img('test_images/'+img)  # this is a PIL image
          x = img_to_array(img)
          x = preprocess_input(x)
          #print (x.shape)
          #x = cv2.cvtColor(y, cv2.COLOR_BGR2RGB)
          #x = x/127.5 -1.
          img_list.append(x)
      #print ('img_list=', len(img_list))
      testing_data = np.array(img_list)

      #print ('testing_data_shape', testing_data.shape)
      #train_data =  train_data[:,:,:,np.newaxis]
      #print (testing_data.shape)
      pred_temp = first_prediction(model_1,testing_data)

      print('batch = ', j, 'stage 1 completed', 'shape=', pred_temp.shape)
      
      
      pred_xgb = xgb1.predict(pred_temp)
      
      
      np.save(src+'temp_test_'+str(j)+'.npy',pred_xgb)

      print ('bacth npy saved', j)

      final_array.append(pred_temp)
      print ('afterbatch',j,'length=', len(final_array))
  final_array = np.array(final_array)
  print ('totally done')

starts doing aata
batch =  0 stage 1 completed shape= (500, 1000)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


bacth npy saved 0
afterbatch 0 length= 1
batch =  1 stage 1 completed shape= (500, 1000)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


bacth npy saved 1
afterbatch 1 length= 2
batch =  2 stage 1 completed shape= (500, 1000)


...

bacth npy saved 18
afterbatch 18 length= 19
batch =  19 stage 1 completed shape= (500, 1000)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


bacth npy saved 19
afterbatch 19 length= 20
batch =  20 stage 1 completed shape= (142, 1000)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


bacth npy saved 20
afterbatch 20 length= 21
totally done


In [69]:

#np.save(src+'data_final',pred_temp)
np.save(src+'data_xgb_train_ensemble.npy',final_array)
print ('file saved ')


file saved 


In [73]:
print (final_array.shape)
test_array = np.concatenate(final_array, axis=0)
test_array.shape


(21,)


(10142, 1000)

In [76]:
prediction = xgb1.predict(test_array)
np.save(src+'xgb_test_prediction.npy',prediction)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [78]:
prediction[:10]

array([1, 2, 5, 0, 1, 5, 5, 5, 0, 2])

In [0]:
test2_df = pd.read_csv('test.csv')
test2_df['Label'] = prediction
test_df['Label'].value_counts()
test2_df.to_csv(src+'test2_xgb.csv',index=False)

In [93]:
print (test2_df["Label"].value_counts())


0    2736
5    2574
1    2043
3    1475
4     928
2     386
Name: Label, dtype: int64


In [91]:
mask = test_df['ID'] != test2_df['ID']
np.sum(mask)

0